In [1]:
# fig = plt.figure(figsize=(13,6))
# plt.title('Close vs Date')
# plt.xlabel('Dates')
# plt.ylabel('Prices')
# plt.plot(train_data['Close'], color='green', label='training_data')
# plt.plot(test_data['Close'], color='red', label='test_data')

In [19]:
# !pip install pmdarima

In [21]:
# from pmdarima.arima import auto_arima
# from pmdarima.arima import ADFTest

# adf_test = ADFTest(alpha=0.05)
# adf_test.should_diff(df['Close'])

In [58]:
# predictions = []
# for i in range(len(test_data)):
#     model = ARIMA(train_data, order=(4,1,0))
#     model_fit = model.fit()
#     output = model_fit.forecast()
#     predictions.append(output[0])
#     train_data.append(test_data[i])

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
# arima_model = auto_arima(train_data['Close'],start_p=0, start_P=0, start_q=0, start_Q=0, d=0, D=0,max_p=5, max_P=5, max_q=5, max_Q=5, max_d=5, max_D=5, m=12, random_state=17, n_fits=10, error_action='warn')

In [22]:
# arima_model.summary()

In [29]:
# test_data.index

Index(['2019-10-27', '2019-10-26', '2019-10-25', '2019-10-24', '2019-10-23',
       '2019-10-22', '2019-10-21', '2019-10-20', '2019-10-19', '2019-10-18',
       ...
       '2019-07-28', '2019-07-27', '2019-07-26', '2019-07-25', '2019-07-24',
       '2019-07-23', '2019-07-22', '2019-07-21', '2019-07-20', '2019-07-19'],
      dtype='object', name='Date', length=101)

In [26]:
# predicted = pd.DataFrame(arima_model.predict(n_periods=test_data.shape[0]), index=test_data.index)
# predicted.head(5)

In [ ]:
# plt.figure(figsize=(16,9))
# plt.xlabel('dates')
# plt.ylabel('prices')
# plt.title('Plottt')
# plt.plot(df[0:int(df.shape[0]*0.9)]['Close'], color='green', label='training_data')
# plt.plot(test_df['Close'], color='red', label='test_data')
# plt.plot(predictions_df['Close'], color='orange', label='predicted_values')
# plt.legend()

In [ ]:
# plt.figure(figsize=(10,6))
# plt.xlabel('dates')
# plt.ylabel('prices')
# plt.title('Plottt')
# # plt.plot(df[0:int(df.shape[0]*0.9)]['Close'], color='green', label='training_data')
# plt.plot(test_df['Close'], color='green', label='test_data')
# plt.plot(predictions_df['Close'], color='red', label='predicted_values')
# plt.legend()

In [56]:
# from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# mean_absolute_percentage_error(test_df['Close'],predictions_df['Close'])

In [8]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import mysql.connector
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error

class Get_Predictions:
    def __init__(self, stock):
        self.host = "firstdatabase.csdx2ljymbci.ap-south-1.rds.amazonaws.com"
        self.port = "3306"
        self.db = "BitCoin_db"
        self.username = "admin"
        self.password = "Admin#123"
        self.stock = stock
        self.cryptos = {'BTC':'BitCoin'}
        

    def connect_to_db(self):
        try:
            connection = mysql.connector.connect(host=self.host,
                                                 user=self.username,
                                                 password=self.password,
                                                 database=self.db)
        except(e):
            raise e
        else:
            print("Connection Successfull")
            return connection
        
    def get_data(self,cursor):
        query = """SELECT * FROM """+self.cryptos[self.stock]+ """;"""
        cursor.execute(query)
        data = cursor.fetchall()
        df = pd.DataFrame(data,columns=['Date','Open','High','Low','Close','Volume'])
        df['Date'] = df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
        df.set_index('Date',inplace=True)
        print(df.dtypes)
#         train_data = df[0:1000]['Close']
#         test_data = df[1000:]['Close']
#         print(train_data.shape, test_data.shape)
        return df['Close']
    
    def create_model(self, no_of_days):
        conn = self.connect_to_db()
        cursor = conn.cursor()
        
        train_data = self.get_data(cursor)
        dates = train_data.index
        temp = list(train_data.values)
#         print(len(temp))
        train_data = list(train_data.values)
        x = int(len(temp) - (10*len(temp))/100)
        print(x)
        temp = temp[x:]
        dates = dates[x:]
#         print(test_data.columns)
#         print(test_data.head(3))
        print("Training...")
        predictions = []
        print(type(train_data))
        for i in range(no_of_days):
            model = ARIMA(train_data, order=(10,0,0))
            model_fit = model.fit()   
            pred = model_fit.forecast(1)
            print(pred)
            predictions.append(pred[0])
            train_data.append(pred[0])
#         predictions_df = pd.DataFrame({'Close':predictions})
#         print(test_data)
#         print(predictions_df)
#         err = mean_absolute_percentage_error(test_data.values,predictions_df['Close'])
        train_data = train_data[x:]
        return (len(dates),len(temp),len(train_data))
    
obj = Get_Predictions('BTC')
print(obj.create_model(3))

Connection Successfull
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object
909
Training...
<class 'list'>
[3021167.71898278]
[3006515.13363306]
[3005681.50446449]
(102, 102, 105)
